# Geology
fields of dream

In [ ]:
#| default_exp geology

### Prior Art

In [ ]:
#| export
#standard
import numpy as np
import sys
import os
import math
import random

#data
from collections import namedtuple, deque
from dataclasses import dataclass,  field, asdict
from typing import List
from enum import Enum

#Jeremy

from fastcore.basics import patch
from fasthtml.common import *
from fasthtml.jupyter import *
import httpx


In [ ]:
#| export


from HexMagic.styles import StyleCSS, SVGBuilder
from HexMagic.primitives import MapCord, MapSize, MapRect, MapPath, Hex, HexGrid, HexWrapper, HexPosition, hexBackground,windy_edge, HexRegion, unique_windy_edge
from HexMagic.terrain import  TerraDemo, Terrain, GeoBounds
from HexMagic.terrainpatterns import TerrainPatterns


In [ ]:
#| export
from HexMagic.voronoi import PlateKind

In [ ]:
#| export
from HexMagic.water.soil import SoilSystem, SoilType
from HexMagic.water.river import River, RiverDemo
from HexMagic.weather import TerraDemo
from HexMagic.water.watershed import Watershed

In [ ]:
#| export
from HexMagic.water.basin import DrainageBasins

In [ ]:
#| export
class Geology:

    def __init__(self, terrain, plates,name="Untitled"): 
        self.plates = plates
        self.update(terrain)
        self.name = name

    def update(self,terrain):
        self.terrain = terrain
        terrain.compute_weather()
        self.soils = SoilSystem.from_plates(terrain,self.plates)
        self.basins = DrainageBasins(terrain)
        self.patterns = TerrainPatterns(terrain)
        

    @classmethod
    def simpleWorld(cls, carve=False, build_deltas=False, debug=False):
        
        # 1. Create blank ocean world with tropical preset
        mySize = MapSize(800, 500)
        myBounds = MapRect(MapCord(0,0), mySize)
        
        t, plates = Terrain.fromSeeds(
            myBounds, 
            radius=15,
            num_plates=16,
            oceanic_sides=['N','W'],
            formation_type='ocean_distance',  # 'ridge', 'volcanic', 'rift', 'rolling'
            # Fine-tuning
            elevation_scale=1.0,
            age='young',  # Fewer subdivisions = clearer boundaries
            seed=42
        )
        


        t.geo = GeoBounds(
            lat_min=20.57,   # Southern tip (near Makena)
            lat_max=21.03,   # Northern tip (near Kahakuloa)
            lon_min=-156.69, # Western tip (West Maui)
            lon_max=-155.97  # Eastern tip (Haleakalā/Hāna)
        )

        # Compute hex coordinates
        #t._compute_hex_coordinates()
        t.climate = TerrainPatterns(t).weatherPatterns()["mediterranean"]
        world = Geology(t,plates,"Hexland")

        terrain = world.terrain
        
        if carve:
            rivers = terrain.carve_to_ocean(num_lakes=1)
        
        # Build deltas from major watersheds
        if build_deltas:
            world.update(terrain)
            
            major_watersheds = world.basins.get_major(top_n=5)
            if debug:
                print(f"building sheds {major_watersheds}")
            
            for watershed in major_watersheds:
                if watershed.is_ocean:  # Only ocean-draining watersheds get deltas
                    if debug:
                        print(f"building delta {watershed.terminal_hex}")
                    world.terrain.buildUp(watershed,rings=5,ele=7)
                else:
                    if debug:
                        print(f"does not go to ocean {watershed.terminal_hex}")

            if carve:
                rivers = watershed.terrain.carve_to_ocean(num_lakes=1)



        world.update(world.terrain.shrinkWeather(0.75))
        world.terrain.geo = GeoBounds(
            lat_min=20.57,   # Southern tip (near Makena)
            lat_max=21.03,   # Northern tip (near Kahakuloa)
            lon_min=-156.69, # Western tip (West Maui)
            lon_max=-155.97  # Eastern tip (Haleakalā/Hāna)
        )

        return world


In [ ]:
#| export
@patch
def baseMap(self:Geology,showHexes=True):


    grid = self.terrain.hexGrid
    builder = grid.builder
    builder.layers = []
    self.terrain.colorMap()
    if showHexes:
        grid.update()
    else:
        builder.adjust("elevations", grid.styleLayerOrdered(
        styles=self.terrain.colorLevels,
        f=unique_windy_edge(iterations=3)))

    builder.adjust("watersheds", self.basins.draw_watersheds())
    builder.adjust("legend",builder.legendOverlay(self.terrain.colorLevels,width=100))

    legend_text = f"{self.name}"
    
    self.terrain.hexGrid.builder.add_centered_text(
        legend_text, 
        y_offset=-self.terrain.hexGrid.builder.height/2 + 30,
        class_name="watershed_legend"
    )

    return builder.show()

In [ ]:
Geology.simpleWorld().baseMap()

In [ ]:
Geology.simpleWorld(carve=True,debug=True).baseMap(showHexes=False)

Done at iter 1: 1 lakes


In [ ]:
Geology.simpleWorld(carve=True,build_deltas=True,debug=True).baseMap()

Done at iter 1: 1 lakes


building sheds [<HexMagic.water.watershed.Watershed object>, <HexMagic.water.watershed.Watershed object>, <HexMagic.water.watershed.Watershed object>, <HexMagic.water.watershed.Watershed object>, <HexMagic.water.watershed.Watershed object>]
building delta 1355
building delta 392
building delta 1705
building delta 1172
building delta 1621
Done at iter 1: 1 lakes


In [ ]:
#| export
@patch
def weatherMap(self:Geology):

    grid = self.terrain.hexGrid
    builder = grid.builder
    builder.layers = []
    self.terrain.colorMap()
    builder.adjust("icon",self.terrain.render_icon_temperature())
    #smaller.hexGrid.update()

    self.terrain.add_rain_overlay()
    builder.adjust("watersheds", self.basins.draw_watersheds())

    legend = []
    legend.append(StyleCSS("Very_Cold",fill="#8B4789"))
    legend.append(StyleCSS("Cold",fill="#4A90E2"))
    legend.append(StyleCSS("Cool",fill="#50C878"))
    legend.append(StyleCSS("Warm",fill="#F4D03F"))
    legend.append(StyleCSS("Hot",fill="#E74C3C"))
    for style in legend:
        builder.add_style(style)
    builder.adjust("legend",builder.legendOverlay(legend,width=100))

    
    return builder.show()

In [ ]:
Geology.simpleWorld().weatherMap()

In [ ]:
#| export
@patch
def flowMap(self:Geology,showHexes=True):

    grid = self.terrain.hexGrid
    builder = grid.builder
    builder.layers = []
    self.terrain.colorMap()
    if showHexes:
        grid.update()
    else:
        builder.adjust("elevations", grid.styleLayerOrdered(
        styles=self.terrain.colorLevels,
        f=unique_windy_edge(iterations=3)))
        
    builder.adjust("flow_diagram",self.terrain.flow_diagram())
    #self.terrain.add_rain_overlay()
    builder.adjust("watersheds", self.basins.draw_watersheds())

    legend_text = f"{self.name} Gradient"
    
    self.terrain.hexGrid.builder.add_centered_text(
        legend_text, 
        y_offset=-self.terrain.hexGrid.builder.height/2 + 30,
        class_name="watershed_legend"
    )

    return builder.show()

In [ ]:
Geology.simpleWorld(carve=True).flowMap(showHexes=False)

Done at iter 1: 1 lakes


In [ ]:
#| export
@patch
def soilMap(self:Geology):

    grid = self.terrain.hexGrid
    builder = grid.builder
    builder.layers = []
    builder.adjust("soilInformation", self.soils.soilOverlay(f=unique_windy_edge(iterations=2)))

    cols = [x.to_nc() for x in SoilType.standard_types()]
    legend = [StyleCSS(x.name,fill=x.color) for x in cols]
    for style in legend:
        builder.add_style(style)
    builder.adjust("legend",builder.legendOverlay(legend,width=100))
    legend_text = f"{self.name} Soil"
    
    self.terrain.hexGrid.builder.add_centered_text(
        legend_text, 
        y_offset=-self.terrain.hexGrid.builder.height/2 + 30,
        class_name="watershed_legend"
    )


    return builder.show()

In [ ]:
Geology.simpleWorld(carve=True).soilMap()

Done at iter 1: 1 lakes


In [ ]:
#| export
@patch
def watershedMap(self:Geology,showHexes=False):

    grid = self.terrain.hexGrid
    builder = grid.builder
    builder.layers = []
   
    self.terrain.colorMap()
    #builder.adjust("icon",self.terrain.render_icon_temperature())
    if showHexes:
        grid.update()
    else:
        builder.adjust("elevations", grid.styleLayerOrdered(
        styles=self.terrain.colorLevels,
        f=unique_windy_edge(iterations=3)))

    builder.adjust("flow_diagram",self.terrain.flow_diagram())
    builder.adjust("watersheds", self.basins.dotted_watershed_overlay(min_density=0.5))
    builder.adjust("watershed_boundaries", self.basins.boundary_overlay())

    legend_text = f"{self.name} has {len(self.basins.sheds)} Drainage Basins"
    
    self.terrain.hexGrid.builder.add_centered_text(
        legend_text, 
        y_offset=-self.terrain.hexGrid.builder.height/2 + 30,
        class_name="watershed_legend"
    )

    return builder.show()
    #terrain.hexGrid.builder.adjust("borders",terrain.elevation_borders())

In [ ]:
Geology.simpleWorld(carve=True).watershedMap()

Done at iter 1: 1 lakes
